In [7]:
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

False

In [8]:
# generate system prompt
class SystemPrompt:
    def __init__(self) -> None:
        pass

    def generate_system_context(self):
        system_context = ("""You are an AI-powered exam conductor responsible for creating and administering tests on a defined subject.  
            Your goal is to generate structured exams with varying difficulty levels, ensuring a balanced assessment of the student's knowledge.  
            The test should include a mix of question types, such as:  

            - **Multiple Choice Questions (MCQs)**  
            - **True/False Questions**  
            - **Fill in the Blanks**  
            - **Reasoning and Analytical Questions**  
            - **Paragraph-based Questions**  
            - **Short and Long Essays**  

            Adapt the difficulty level based on the student's proficiency and provide clear instructions for each section.  
            Ensure the questions are diverse, fair, and encourage critical thinking while maintaining subject relevance.  

            {context}
            """)
        
        return system_context

In [9]:
# embedder class
class GetEmbedder:
    def __init__(self):
        pass

    def embedding_instance_provider(self):
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        return embeddings

In [10]:
# configuring Google Gemini API
apiKey = ""

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = apiKey

print("Gemini API Key set successfully!")

# configuring PineCone API Credentials
pineconeKey = ""

if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = pineconeKey

print("PineCone API Key set successfully!")

Gemini API Key set successfully!
PineCone API Key set successfully!


In [15]:
# 🔹 Pinecone Connector Class
class PineConeConnector:
    def __init__(self, uid, embedder) -> None:
        self.uid = uid
        self.embedder = embedder
        self.index_name = f"{self.uid}"

        # Initialize Pinecone once
        self.pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

    def get_retriever(self):
        """Retrieves stored embeddings from Pinecone."""
        doc_store = PineconeVectorStore.from_existing_index(
            index_name=self.index_name,
            embedding=self.embedder,
        )
        retriever = doc_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        return retriever

In [16]:
index_name = "teacher-bot-test"
embedder = GetEmbedder().embedding_instance_provider()

print("Embedder and index name have been set")

Embedder and index name have been set


In [17]:
pinecone_instance = PineConeConnector(uid=index_name, embedder=embedder)

print('Pinecone has been connected')

Pinecone has been connected


In [18]:
retriever = pinecone_instance.get_retriever()

In [21]:
# 🔹 Gemini RAG Class
class GeminiRag:
    def __init__(self, retriever, question):
        self.retriever = retriever
        self.question = question

        # Initialize LLM
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.9,)

        # Generate system context and set up prompt template
        system_context = SystemPrompt().generate_system_context()
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_context),
                ("human", "{input}"),
            ]
        )

    def answer_prompt(self):
        """Retrieves documents and answers the prompt."""
        prompt_answer_chain = create_stuff_documents_chain(self.llm, self.prompt)
        rag_chain = create_retrieval_chain(self.retriever, prompt_answer_chain)

        return rag_chain.invoke({"input": self.question})

In [24]:
question = "100 mark test on BioPolymers with 10 2 marks, 5 13 marks and 1 16 marks question pattern, difficult of the test: Hard"

testgenerator_instance = GeminiRag(retriever, question)

In [25]:
response = testgenerator_instance.answer_prompt()

In [26]:
print(response)

{'input': '100 mark test on BioPolymers with 10 2 marks, 5 13 marks and 1 16 marks question pattern, difficult of the test: Hard', 'context': [Document(id='e39a2e9c-3897-4318-89de-96b1ba7cd203', metadata={'author': 'Acer', 'creationdate': '2024-04-20T12:19:22+05:30', 'creator': 'Microsoft® Word 2010', 'moddate': '2024-04-20T12:19:22+05:30', 'page': 0.0, 'page_label': '1', 'producer': 'Microsoft® Word 2010', 'source': 'data\\text_book.pdf', 'total_pages': 4.0}, page_content='\uf0b7 On the other hand if the backbone chains can be made more rigid then a stiffe r \npolymer will result.'), Document(id='8cc23816-096b-4deb-99f1-36f4ef8ddb95', metadata={'author': 'Acer', 'creationdate': '2024-04-20T12:19:22+05:30', 'creator': 'Microsoft® Word 2010', 'moddate': '2024-04-20T12:19:22+05:30', 'page': 2.0, 'page_label': '3', 'producer': 'Microsoft® Word 2010', 'source': 'data\\text_book.pdf', 'total_pages': 4.0}, page_content='3 \n \n \n\uf0b7 Since polymers are noncrystalline or at most semicrysta

In [23]:
class ExamCorrectionSystemPrompt:
    def __init__(self):
        pass

    def generate_system_context(self):
        """Defines the AI’s role as an exam evaluator."""
        system_context = ("""You are an AI-powered examination correction assistant responsible for evaluating students' answers and grading them fairly.  
            Your goal is to assess responses based on accuracy, depth, reasoning, and adherence to the question's requirements.  
            
            **Evaluation Criteria:**  
            - **Correctness**: Ensure the answer accurately addresses the question.  
            - **Relevance**: Verify that the response stays on topic.  
            - **Clarity & Explanation**: Check for well-structured and logical explanations.  
            - **Depth of Understanding**: Award higher marks for in-depth reasoning.  
            - **Grammar & Presentation**: Consider clarity in writing and coherence.  

            **Scoring System:**  
            - **MCQs & True/False:** Assign 1 or 0 based on correctness.  
            - **Short Answers:** Award partial marks for partially correct responses.  
            - **Essays & Explanations:** Grade based on argument strength, structure, and depth.  

            Provide a clear **score breakdown** and **feedback** for each response to help students improve.  

            {context}
        """)
        return system_context


In [27]:
class GeminiExamCorrection:
    def __init__(self, answers, retriever):
        self.answers = answers
        self.retriever = retriever

        # Initialize LLM
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.9)

        # Generate system context and set up prompt template
        system_context = ExamCorrectionSystemPrompt().generate_system_context()
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_context),
                ("human", "{input}"),
            ]
        )

    def correct_exam(self):
        """Retrieves answer-related documents and evaluates responses."""
        correction_chain = create_stuff_documents_chain(self.llm, self.prompt)
        rag_chain = create_retrieval_chain(self.retriever, correction_chain)

        return rag_chain.invoke({"input": self.answers})

In [28]:
sample_input = """
Question 1:
What are biopolymers?
Answer: Biopolymers are synthetic polymers that are created in chemical laboratories.

Question 2:
Give an example of a polysaccharide biopolymer.
Answer: DNA is an example of a polysaccharide biopolymer.

Question 3:
What is the primary function of proteins as biopolymers?
Answer: Proteins mainly store genetic information and transmit hereditary traits.

Question 4:
How are biopolymers different from synthetic polymers?
Answer: Biopolymers and synthetic polymers have no significant differences; both are man-made materials.

Question 5:
What is the monomer unit of cellulose?
Answer: The monomer unit of cellulose is amino acid.
"""

corrector_rag = GeminiExamCorrection(retriever=retriever, answers=sample_input)

In [29]:
response = corrector_rag.correct_exam()

In [30]:
print("Response")
print(response)

Response
{'input': '\nQuestion 1:\nWhat are biopolymers?\nAnswer: Biopolymers are synthetic polymers that are created in chemical laboratories.\n\nQuestion 2:\nGive an example of a polysaccharide biopolymer.\nAnswer: DNA is an example of a polysaccharide biopolymer.\n\nQuestion 3:\nWhat is the primary function of proteins as biopolymers?\nAnswer: Proteins mainly store genetic information and transmit hereditary traits.\n\nQuestion 4:\nHow are biopolymers different from synthetic polymers?\nAnswer: Biopolymers and synthetic polymers have no significant differences; both are man-made materials.\n\nQuestion 5:\nWhat is the monomer unit of cellulose?\nAnswer: The monomer unit of cellulose is amino acid.\n', 'context': [Document(id='8cc23816-096b-4deb-99f1-36f4ef8ddb95', metadata={'author': 'Acer', 'creationdate': '2024-04-20T12:19:22+05:30', 'creator': 'Microsoft® Word 2010', 'moddate': '2024-04-20T12:19:22+05:30', 'page': 2.0, 'page_label': '3', 'producer': 'Microsoft® Word 2010', 'source